In [ ]:
# this note book is for exporting the images from nicfi and sentinel datasource


# the exporting rules:
# - Target shapefile: 1823_ADRSM
# - Important field:'DEN_BOT','AREA_HA','Index',"SHAPE@"
# - Only export the AREA_HA > 10
# - nicfi datasource imported by gee in python notebook
# - Sentinel2 could imported by several channel , in this solution is Using arcgis pro Living Atlas (AWS host)
# - Dynamic scale properties: 
#                                   size <10ha , image will scall at 8times 
#                             the shape 10 ha<size < 50ha , image will scall at 5times.
#                            - 50ha<shape size<100ha,  image will scall at 2 times
                             #- shape size> 100ha, image will scall at 1.3times

# Five Years and 60 image for each dumpsite and using nicif 
# 

# Usage:  
# 1. Each block have the function description
# 2. Nicf: - Get nicif
        #  - Determin the date range ( check the new update)
        #  - Get Project , map
        #  - Processing Exporting
        #  - 
# 3. Sentinel: - Add sentinel data from Livingatlas( or other resoucre option )

# 4. Export the 5 years each 12 month of image,60total for one shape( nicfi) 

# Environment:
# - Arcgis pro 3.2.2
# - Python 3.8.19

In [60]:
# lib import 

import arcpy
# check the shapefile return selected count of feature class
print(arcpy.GetCount_management("1823_ADRSM")) # if selected 1 it will return 1

print("Import, and load shape")

1823
Import, and load shape


In [61]:
# Get nicif

# add gee into arcgispro
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-qinheyi')

# init the map
m=geemap.Map()

nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

def getDateRange(imageCollection:ee.ImageCollection):
    dateRange=imageCollection.aggregate_array('system:time_start')
    date_list=[ee.Date(date).format().getInfo() for date in dateRange.getInfo()]
    print(date_list)
    return date_list

# imageRange=getDateRange(nicfi) # because of the server-side function.

baseMap=nicfi.filter(ee.Filter.date('2024-02-01','2025-01-11')).sort('system:time_start', False).first()

print("basemap loaded")

basemap loaded


In [68]:
# Add nicif

# m.centerObject(baseMap,4)
# vis={'bands':['R','G','B'],'min':64,'max':5454,'gamma':1.8}  # original vis
vis={'bands':['R','G','B'],'min':500,'max':3500,'gamma':2.0}
m.addLayer(baseMap,vis,'nicfi')
m.height='1600px'
display(m)

Map(bottom=2316683.0, center=[-18.479202030551182, 289.72088813781744], controls=(WidgetControl(options=['posi…

In [22]:
# check out the nicfi time list to decide the date range
selected_data='2024-03-01'

filted_collection=nicfi.filter(ee.Filter.date(selected_data,'2025-01-11')).sort('system:time_start', False)
# Get the dates of all images in the collection
dates = filted_collection.aggregate_array('system:time_start')
# Convert dates to a list
dates_list = dates.getInfo()
# Iterate over the dates and print them out
print(len(dates_list))
for date in dates_list:
    print("Date:", ee.Date(date).format("YYYY-MM-dd").getInfo())


1
Date: 2024-03-01


In [32]:
# Get Project, map, mapview

current_project=arcpy.mp.ArcGISProject("CURRENT")
layout=current_project.listLayouts()[1]  # 0 is the dataframe drive layout(text,area) # 1 is the 
print("current layout name:",layout.name)

amap=current_project.listMaps()[0]

mv=current_project.activeView    #mv=amap.defaultView

print(mv.camera.getExtent ())
print(amap.name)

current layout name: JpegExport
-8475310.2467247 -1503644.91400562 -8470258.71134624 -1499351.47340258 NaN NaN NaN NaN
Map


In [61]:
# Exporting 
# Before exporting, confirm the layout is setting well


# get the certain shape of shapefile  and iterate the shapefile
# handle the mv (mapview), do not effect the layout 
import time 
start_time = time.time()

fc = "1823_ADRSM"
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]

#get the mapframe in the layout
mf= layout.listElements("MAPFRAME_ELEMENT")[0]

path=r"C:\Users\lycaz\Desktop\JPEGS_sent"    # change it before exporting !!!!!!
time_stamp="202403"                         # check it as well 
text_datasource="Sentinal" # or "Nicif" "Sentinel"        # change it bfore export 




with arcpy.da.SearchCursor(fc, fields) as cursor:
    count_im=0
    for row in cursor:
        #print(u'{0}, {1},{2}'.format(row[2],row[0], row[1]))
     
        print(f" Processing index {row[2]} {row[0]}", end="\r")
        if row[1] > 10:
            count_im+=1
            print(f"image coount {count_im}", end="\r")
            geometry=row[3]
            mv.camera.setExtent(geometry.extent)
            amap.defaultCamera=mv.camera
            amap.defaultCamera.scale*=1.5
            mf.camera.setExtent(amap.defaultCamera.getExtent())
            if row[1]<10:
                mf.camera.scale*=8
            elif row[1]<20:
                mf.camera.scale*=5
            elif row[1]<100:
                mf.camera.scale*=2
            else:
                mf.camera.scale*=1.3
            ele=layout.listElements(wildcard="Text")[0]
            ele.text=f"{row[2]}-{row[0]}-{time_stamp}-{row[1]:.2f}ha"
            ele_source=layout.listElements(wildcard="Text 1")[0]
            ele_source.text=text_datasource
            layout.exportToJPEG(f"{path}\\{str(row[2])}-{time_stamp}-{text_datasource}.jpg",resolution=300)
    print('\n')
    print("Complete all the image")

stop_time = time.time()
elapsed_time = stop_time - start_time

print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete all the image
Start time: Sat Apr 13 09:25:18 2024
Stop time: Sat Apr 13 09:30:00 2024
Elapsed time: 282.59 seconds


In [31]:
# Function for timely review the imagry
# iterate all time from recently 5 years 
# show it in certain place 



# function to query certain shape return the geometrys , using index ,using size ha as query_str
def query_geometry(query_str):
    geometries = []
    # Use a search cursor to iterate over the features that match the query
    with arcpy.da.SearchCursor("1823_ADRSM", ['AREA_HA','SHAPE@'], where_clause=query_str) as cursor:
        for row in cursor:
            # Access the geometry of each feature
            geometry = row[1]
            # Add the geometry to the list
            geometries.append(geometry)
            print(row[0])
            print(type(row[1]))
    return geometries


# showcase 

sql_query="Index=1171"
selected_geo=query_geometry(sql_query)
if len(selected_geo)==1:
    print(selected_geo)
    print(selected_geo[0].extent.height,selected_geo[0].extent.width)
    # center to this geo
    #mv.panToExtent(selected_geo[0].extent)
    #mv.camera.setExtent(selected_geo[0].extent)
    amap.defaultCamera.setExtent(selected_geo[0].extent)
    amap.defaultCamera.scale*=1.5
    
    print(mv.camera.getExtent ())

    

# change basemap

start_year=2019
end_year=2024
date_list=[]
for i in range(start_year,end_year):
    for j in range(3,13):
        if j >9:
            date_list.append(f"{i}-{j}-01")
        else:
            date_list.append(f"{i}-0{j}-01")
    date_list.append(f"{i+1}-01-01")
    date_list.append(f"{i+1}-02-01")


basemaps=       
    

181.24151696
<class 'arcpy.arcobjects.geometries.Polygon'>
[<Polygon object at 0x1f38ab44070[0x1f30ed3fde0]>]
0.025418119999999433 0.019702124999994908
-76.116801368 -13.3816299269999 -76.097099243 -13.3562118069999 0 0 nan nan


In [82]:
from geemap.ee_tile_layers import _get_tile_url_format

basemap=nicfi.filter(ee.Filter.date('2019-01-01','2023-03-11')).sort('system:time_start', False).first()

print(type(basemap))
# Define visualization parameters
vis={'bands':['R','G','B'],'min':500,'max':3500,'gamma':2.0}

# Get the tile URL format
map_id_dict = ee.Image(basemap).getMapId(vis)
print(map_id_dict)
url = map_id_dict["tile_fetcher"].url_format
print(url)

url_format = _get_tile_url_format(basemap, vis)
print(url_format)

<class 'ee.image.Image'>
{'mapid': 'projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-4115f79a30eba797b6c398402fa88132', 'token': '', 'tile_fetcher': <ee.data.TileFetcher object at 0x000001F39CC21070>, 'image': <ee.image.Image object at 0x000001F39CC4F4F0>}
https://earthengine.googleapis.com/v1/projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-4115f79a30eba797b6c398402fa88132/tiles/{z}/{x}/{y}
https://earthengine.googleapis.com/v1/projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-e42174e0132e6abec57d4116f009c856/tiles/{z}/{x}/{y}
